In [43]:
import numpy as np
import pandas as pd
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

### Merge Wodify, SugarWOD and Strava into one table

Starting where data_discovery_manipulation notebook left off, merge full data tables

In [44]:
strava = pd.read_json('../observations/strava/activities.json')
sugarwod = pd.read_csv('../observations/sugarwod/workouts.csv')
wodify_lifts = pd.read_excel('../observations/wodify/PerformanceResults.xlsx')
wodify_metcons = pd.read_excel('../observations/wodify/PerformanceResultsMetcons.xlsx')
wodify_prs = pd.read_excel('../observations/wodify/PerformanceResultsPRs.xlsx')

#### Make a 'date' column in each Dataframe of the same format 

In [45]:
strava['start_date_local'] = pd.to_datetime(strava['start_date_local'])
strava['start_time'] = strava['start_date_local'].dt.strftime('%H:%M:%S')

strava['date'] = strava['start_date_local'].dt.strftime('%Y-%m-%d')
strava = strava.drop(columns=['start_date_local'])

strava['strava_name'] = strava['name']
strava = strava.drop(columns=['name'])

sugarwod['date'] = pd.to_datetime(sugarwod['date'])
sugarwod['date'] = sugarwod['date'].dt.strftime('%Y-%m-%d')

wodify_lifts['date'] = wodify_lifts['Date'].dt.strftime('%Y-%m-%d')
wodify_lifts = wodify_lifts.drop(columns=['Date'])

wodify_metcons['date'] = wodify_metcons['Date'].dt.strftime('%Y-%m-%d')
wodify_metcons = wodify_metcons.drop(columns=['Date'])

wodify_prs['Peformance Result Date'] = pd.to_datetime(wodify_prs['Performance Result Date'])
wodify_prs['date'] = wodify_prs['Performance Result Date'].dt.strftime('%Y-%m-%d')
wodify_prs = wodify_prs.drop(columns=['Performance Result Date'])

#### Rename duplicate columns names where merges would result in data loss

In [46]:
wodify_lifts['lift_component'] = wodify_lifts['Component']
wodify_lifts = wodify_lifts.drop(columns=['Component'])

wodify_metcons['metcon_component'] = wodify_metcons['Component']
wodify_metcons = wodify_metcons.drop(columns=['Component'])

wodify_prs['pr_component'] = wodify_prs['Component Name']
wodify_prs = wodify_prs.drop(columns=['Component Name'])

wodify_lifts['is_lift_pr'] = wodify_lifts['Is Personal Record']
wodify_lifts['lift_pr_description'] = wodify_lifts['Personal Record Description']
wodify_lifts = wodify_lifts.drop(columns=['Is Personal Record'])
wodify_lifts = wodify_lifts.drop(columns=['Personal Record Description'])

wodify_metcons['is_metcon_pr'] = wodify_metcons['Is Personal Record']
wodify_metcons['metcon_pr_description'] = wodify_metcons['Personal Record Description']
wodify_metcons = wodify_metcons.drop(columns=['Is Personal Record'])
wodify_metcons = wodify_metcons.drop(columns=['Personal Record Description'])

#### Text describing the CrossFit workout is in the 'description' column on SugarWOD and the 'Component Description' of Wodify

#### Rename these so they will merge on concatenation: this is safe so long as there is one WOD performed per day

#### Rethink if there becomes a case where two different WODs are performed on the same day on different apps

In [48]:
wodify_metcons['wod'] = wodify_metcons['Component Description']
wodify_metcons = wodify_metcons.drop(columns=['Component Description'])

sugarwod['wod'] = sugarwod['description']
sugarwod = sugarwod.drop(columns=['description'])

#### Merge the dataframes on 'date' attribute: All the workout data for one day in each row

In [49]:
wodify_all = pd.concat([wodify_lifts, wodify_metcons, wodify_prs])

all_data = pd.concat([wodify_all, strava, sugarwod])

all_data.loc[all_data['date'] == '2023-01-10']

,Affiliate Name,Class Name,Result,Performance Result Type,Comment,From Weightlifting Total,date,lift_component,is_lift_pr,lift_pr_description,...,strava_name,title,best_result_raw,best_result_display,score_type,barbell_lift,set_details,notes,rx_or_scaled,pr
4,CrossFit Cove,Cove Fitness: Noon,1 x 5 @ 275 lbs,Weight,NaN,False,2023-01-10,Deadlift,True,PR by 20 lbs vs. 255 on 11/02/2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,CrossFit Cove,Cove Fitness: Noon,NaN,NaN,NaN,NaN,2023-01-10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Cove Fitness: Noon,1 x 5 @ 275 lbs,NaN,NaN,NaN,2023-01-10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
all_data_by_date = all_data.groupby(all_data['date'], as_index=False).aggregate('first')

all_data_by_date.loc[all_data_by_date['date'] == '2023-01-10']

,date,Affiliate Name,Class Name,Result,Performance Result Type,Comment,From Weightlifting Total,lift_component,is_lift_pr,lift_pr_description,...,strava_name,title,best_result_raw,best_result_display,score_type,barbell_lift,set_details,notes,rx_or_scaled,pr
219,2023-01-10,CrossFit Cove,Cove Fitness: Noon,1 x 5 @ 275 lbs,Weight,NaN,False,Deadlift,True,PR by 20 lbs vs. 255 on 11/02/2022,...,None,None,NaN,None,None,None,None,None,None,None


#### There are 87 columns in the full merge data set. How do we verify them all?

In [51]:
all_data_by_date.columns

Index(['date', 'Affiliate Name', 'Class Name', 'Result',
       'Performance Result Type', 'Comment', 'From Weightlifting Total',
       'lift_component', 'is_lift_pr', 'lift_pr_description', 'Component(2)',
       'Fully Formatted Result', 'Is Rx', 'Is Rx Plus', 'Result Type Label',
       'Full Comment', 'metcon_component', 'is_metcon_pr',
       'metcon_pr_description', 'wod', 'Rep Scheme',
       'Performance Result Comment', 'Personal Record Text',
       'Peformance Result Date', 'pr_component', 'resource_state', 'athlete',
       'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain',
       'type', 'sport_type', 'id', 'start_date', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'map', 'trainer', 'commute', 'manual', 'private',
       'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng',
       'average_speed', 'max_sp

In [52]:
all_data_by_date['start_date']

0      2021-07-26T21:01:22Z
1      2021-08-04T21:35:51Z
2      2021-08-10T21:08:33Z
3      2021-08-14T11:36:56Z
4      2021-08-18T20:56:44Z
               ...         
228                    None
229                    None
230                    None
231                    None
232                    None
Name: start_date, Length: 233, dtype: object

In [53]:
all_data_by_date['start_time']

0      17:01:22
1      17:35:51
2      17:08:33
3      07:36:56
4      16:56:44
         ...   
228        None
229        None
230        None
231        None
232        None
Name: start_time, Length: 233, dtype: object

#### TODO: Work to be done sorting out how to handle time: start time, end time, duration

#### TODO: More processing on the 'wod' column to determine what the workout is

In [ ]:
all_data_by_date['wod']